In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import torch
import pandas as pd
import numpy as np
import pickle

In [ ]:
data = pd.read_pickle('./speed_02_000.pkl') ## You may use any speed pickle file generated from Preprocessing2, since only the network information is utilized from the data.

In [ ]:
def DistanceAdj(dataframe, edge_number):
    # initializing the matrix
    real_distance_adj = torch.zeros(edge_number, edge_number)
    
    # EdgeID list generation
    EdgeID_list = list(dataframe.EdgeID.unique())
    
    # Updating the conenctivity adjacency matrix
    for idx in EdgeID_list:
        in_num = dataframe.Link.loc[idx][0]
        out_num = dataframe.Link.loc[idx][1]
        
        for i in range(edge_number):
            if dataframe.Link.iloc[i][0] == out_num:
                if dataframe.Link.iloc[i][1] == in_num:
                    real_distance_adj[idx, i] = 0
                else:
                    real_distance_adj[idx, i] = (dataframe.loc[idx].length + dataframe.iloc[i].length) / 2
            else:
                pass
        real_distance_adj[idx, idx] = 0
        
    return real_distance_adj

In [ ]:
number = len(data_ID)
distance_adj = DistanceAdj(data, number)
distance_df = pd.DataFrame(distance_adj.numpy())
distance_df

In [ ]:
network_graph = {}
for idx1 in EdgeID_list:
    sample = {}
    for idx2 in EdgeID_list:
        if distance_adj[idx1][idx2] != 0:
            sample[idx2] = distance_adj[idx1][idx2]
        else:
            pass
    network_graph[idx1] = sample

In [ ]:
## Code reference: https://justkode.kr/algorithm/python-dijkstra ##

import heapq  # For implementing a priority queue (built-in module)

def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    queue = []
    heapq.heappush(queue, [distances[start], start])
    
    while queue:  # Continue until all nodes are processed
        current_distance, current_destination = heapq.heappop(queue)
        
        if distances[current_destination] < current_distance:
            continue
            
        for new_destination, new_distance in graph[current_destination].items():
            distance = current_distance + new_distance
            if distance < distances[new_destination]:
                distances[new_destination] = distance
                heapq.heappush(queue, [distance, new_destination])
    
    return distances

In [ ]:
dijkstra_df = pd.DataFrame()

for idx in EdgeID_list:
    result = dijkstra(network_graph, idx)
    sample_df = pd.DataFrame.from_dict([result])
    dijkstra_df = pd.concat([dijkstra_df, sample_df])
    
dijkstra_df

In [ ]:
dijkstra_df_copy = dijkstra_df.copy()
dijkstra_df_copy = dijkstra_df_copy.applymap(lambda x: x.item() if torch.is_tensor(x) else x)
dijkstra_df_copy = dijkstra_df_copy.replace(float('inf'), 0.0)

In [ ]:
dijkstra_df_copy.to_csv('../../input_data/dijkstra_matrix.csv', index=False)